In [10]:
import sys
sys.path.append('../')
from memory import DatabaseStorage
from crud.insert import Insert
from crud.update import Update
from crud.delete import Delete
from queries.queryhandler import QueryHandler

In [11]:
import string
import random
import time

In [12]:
def construct_payload(n: int):
    i = 0
    while i < n:
        payload = {
            "name": ''.join(random.choices(string.ascii_letters, k=7)),
            "address": ''.join(random.choices(string.ascii_letters + string.digits, k=30)),
            "massive payload": ''.join(random.choices(string.ascii_letters, k=5000))
        }
        yield payload
        i += 1

### Group Inserts

In [13]:
db = DatabaseStorage(database_location='./', database_name='CS541DB', collection_name='LoadTest1')

In [14]:
ins = Insert(interval = 1)
start = time.time()

for payload in construct_payload(1000):
    ins.insert_one(database='CS541DB', collection_name='LoadTest1', payload=payload, group_commit=False)
end = time.time()
print('The time to perform this insert operation without group commit ', end - start)

The time to perform this insert operation without group commit  5.399606943130493


In [15]:
db = DatabaseStorage(database_location='./', database_name='CS541DB', collection_name='LoadTest2')

In [16]:
ins = Insert(interval = 1)
start = time.time()

for payload in construct_payload(1000):
    ins.insert_one(database='CS541DB', collection_name='LoadTest2', payload=payload, group_commit=True)
ins.stop_commit()
end = time.time()
print('The time to perform this insert operation with group commit ', end - start)

Commit complete
The time to perform this insert operation with group commit  0.5276479721069336


### Group Updates

In [17]:
def construct_payload2(n: int):
    i = 0
    while i < n:
        payload = {
            "_id": "abcdef12341234",
            "content" : ''.join(random.choices(string.ascii_letters, k=5000))
        }
        yield payload
        i += 1

In [18]:
db = DatabaseStorage(database_location='./', database_name='CS541DB', collection_name='LoadTest3')
# Inserting 1 record and updating it 1000 times
ins = Insert()
start = time.time()
payload = {
    "_id" : "abcdef12341234",
    "content" : ''.join(random.choices(string.ascii_letters, k=5000))
}
ins.insert_one(database='CS541DB', collection_name='LoadTest3', payload=payload, group_commit=False)
up = Update(interval=1)
# Now updating it 1000 times
for payload in construct_payload2(10000):
    up.update_one(database='CS541DB', collection_name='LoadTest3', payload=payload, group_commit=False)
end = time.time()
print('The time to perform this insert operation without group commit ', end - start)

The time to perform this insert operation without group commit  8.188508033752441


In [19]:
db = DatabaseStorage(database_location='./', database_name='CS541DB', collection_name='LoadTest4')
# Inserting 1 record and updating it 1000 times
ins = Insert()
start = time.time()
payload = {
    "_id" : "abcdef12341234",
    "content" : ''.join(random.choices(string.ascii_letters, k=5000))
}
ins.insert_one(database='CS541DB', collection_name='LoadTest4', payload=payload, group_commit=False)
up = Update(interval=10)
# Now updating it 1000 times
for payload in construct_payload2(10000):
    up.update_one(database='CS541DB', collection_name='LoadTest4', payload=payload, group_commit=True)
up.stop_commit()
end = time.time()
print('The time to perform this insert operation with group commit ', end - start)

Committing
Commit complete
The time to perform this insert operation with group commit  5.7460479736328125
